In [2]:
path = 'clogit.csv'

In [3]:
from collections import OrderedDict
import pylogit as pl
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import scipy

/root/miniconda3/lib/python3.7/site-packages/statsmodels/compat/pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


## 1. Process data

In [4]:
raw_data = pd.read_csv(path)

In [5]:
columns = [item.strip() for item in raw_data.columns]
raw_data.columns = columns

In [6]:
raw_data.head(5)

,MODE,TTME,INVC,INVT,GC,CHAIR,HINC,PSIZE,AASC,TASC,BASC,CASC,HINCA,PSIZEA
0,0,69,59,100,70,0,35,1,1,0,0,0,35,1
1,0,34,31,372,71,0,35,1,0,1,0,0,0,0
2,0,35,25,417,70,0,35,1,0,0,1,0,0,0
3,1,0,10,180,30,0,35,1,0,0,0,1,0,0
4,0,64,58,68,68,0,30,2,1,0,0,0,30,2


In [7]:
# data describe
# HINC: home income
# PSIZE: travel people count
# TTIME: waiting time in station
# INVC: money cost
# INVT: time cost

In [8]:
cols = ['TTME', 'INVC', 'INVT']
modes = ['AIR', 'TRAIN', 'BUS', 'CAR']
new_col = [item1 + '_' + item2 for item1 in cols for item2 in modes]
new_col

['TTME_AIR',
 'TTME_TRAIN',
 'TTME_BUS',
 'TTME_CAR',
 'INVC_AIR',
 'INVC_TRAIN',
 'INVC_BUS',
 'INVC_CAR',
 'INVT_AIR',
 'INVT_TRAIN',
 'INVT_BUS',
 'INVT_CAR']

In [9]:
model_data = pd.DataFrame(columns=[
    'OBS_ID',
    'HINC',
    'PSIZE'
] + new_col + ['y'])

In [10]:
data_AASC = raw_data[raw_data['AASC'] == 1]
data_TASC = raw_data[raw_data['TASC'] == 1]
data_BASC = raw_data[raw_data['BASC'] == 1]
data_CASC = raw_data[raw_data['CASC'] == 1]

In [11]:
def fill_dt(data_frame, item='TTME', mode='AIR'):
    for idx in data_frame.index:
        row = idx//4
        model_data.loc[row, item+'_'+mode] = raw_data.loc[idx, item]

In [12]:
for item in cols:
    fill_dt(data_AASC, item, 'AIR')
    fill_dt(data_TASC, item, 'TRAIN')
    fill_dt(data_BASC, item, 'BUS')
    fill_dt(data_CASC, item, 'CAR')

In [13]:
for idx in range(model_data.shape[0]):
    row = idx*4
    model_data.loc[idx, 'OBS_ID'] = idx
    model_data.loc[idx, 'HINC'] = raw_data.loc[row, 'HINC']
    model_data.loc[idx, 'PSIZE'] = raw_data.loc[row, 'PSIZEA']
    model_data.loc[idx, 'y'] = (raw_data.loc[row+3, 'MODE'] == raw_data.loc[row+3, 'CASC']).astype(int)

In [14]:
model_data = model_data.drop(columns=['TTME_CAR'])
model_data = model_data.set_index('OBS_ID')

In [15]:
model_data.head(5)

,HINC,PSIZE,TTME_AIR,TTME_TRAIN,TTME_BUS,INVC_AIR,INVC_TRAIN,INVC_BUS,INVC_CAR,INVT_AIR,INVT_TRAIN,INVT_BUS,INVT_CAR,y
OBS_ID,,,,,,,,,,,,,,
0,35,1,69,34,35,59,31,25,10,100,372,417,180,1
1,30,2,64,44,53,58,31,25,11,68,354,399,255,1
2,40,1,69,34,35,115,98,53,23,125,892,882,720,1
3,70,3,64,44,53,49,26,21,5,68,354,399,180,1
4,45,2,64,44,53,60,32,26,8,144,404,449,600,1


## 2. Data information

In [16]:
model_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210 entries, 0 to 209
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   HINC        210 non-null    object
 1   PSIZE       210 non-null    object
 2   TTME_AIR    210 non-null    object
 3   TTME_TRAIN  210 non-null    object
 4   TTME_BUS    210 non-null    object
 5   INVC_AIR    210 non-null    object
 6   INVC_TRAIN  210 non-null    object
 7   INVC_BUS    210 non-null    object
 8   INVC_CAR    210 non-null    object
 9   INVT_AIR    210 non-null    object
 10  INVT_TRAIN  210 non-null    object
 11  INVT_BUS    210 non-null    object
 12  INVT_CAR    210 non-null    object
 13  y           210 non-null    object
dtypes: object(14)
memory usage: 24.6+ KB


In [17]:
model_data = model_data.dropna()
model_data = model_data.fillna(0)

In [18]:
model_data['HINC'] = model_data['HINC'].astype(float)

In [19]:
model_data.head(5)

,HINC,PSIZE,TTME_AIR,TTME_TRAIN,TTME_BUS,INVC_AIR,INVC_TRAIN,INVC_BUS,INVC_CAR,INVT_AIR,INVT_TRAIN,INVT_BUS,INVT_CAR,y
OBS_ID,,,,,,,,,,,,,,
0,35.0,1,69,34,35,59,31,25,10,100,372,417,180,1
1,30.0,2,64,44,53,58,31,25,11,68,354,399,255,1
2,40.0,1,69,34,35,115,98,53,23,125,892,882,720,1
3,70.0,3,64,44,53,49,26,21,5,68,354,399,180,1
4,45.0,2,64,44,53,60,32,26,8,144,404,449,600,1


## 3. Single variable analysis

In [20]:
# Discrete variables analysis
crosstab = pd.crosstab(model_data['y'], model_data['PSIZE'])
p = scipy.stats.chi2_contingency(crosstab)[1]
p

0.0024577358937625327

In [21]:
# Continuous variables analysis
logistic = sm.Logit(model_data['y'], model_data['INVT_CAR']).fit()
p = logistic.pvalues['INVT_CAR']
y_predict = logistic.predict(model_data['INVT_CAR'])
AUC = metrics.roc_auc_score(model_data['y'], y_predict)
print('INVT_CAR p value: %.6f' % p)
print('AUC: %.6f' % AUC)

Optimization terminated successfully.
         Current function value: 0.592252
         Iterations 4
INVT_CAR p value: 0.000000
AUC: 0.624256


## 4. Multicollinearity test

In [22]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [23]:
X = model_data[['INVT_' + item for item in modes]]
vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['features'] = X.columns
vif

,VIF Factor,features
0,14.229424,INVT_AIR
1,72.782420,INVT_TRAIN
2,80.279742,INVT_BUS
3,35.003438,INVT_CAR


## 5. LR model building

In [24]:
# variables with high VIF are excluded from the model
X = model_data[['HINC', 'PSIZE', 'TTME_TRAIN', 'INVC_CAR']]
y = model_data['y']

dummies = pd.get_dummies(X['PSIZE'], drop_first=False)
dummies.columns = ['PSIZE_' + str(x) for x in dummies.columns.values]
X = pd.concat([X, dummies], axis=1)

In [25]:
X = X.drop(columns = ['PSIZE', 'PSIZE_4', 'PSIZE_5', 'PSIZE_6'])

In [26]:
X['Intercept'] = 1

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, random_state=1234)

In [28]:
logistic = sm.Logit(y_train, X_train).fit()
logistic.summary2()

Optimization terminated successfully.
         Current function value: 0.509064
         Iterations 6


<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
=================================================================
Model:              Logit            Pseudo R-squared: 0.149     
Dependent Variable: y                AIC:              185.0455  
Date:               2021-07-05 15:39 BIC:              206.9133  
No. Observations:   168              Log-Likelihood:   -85.523   
Df Model:           6                LL-Null:          -100.51   
Df Residuals:       161              LLR p-value:      3.9775e-05
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     6.0000                                       
------------------------------------------------------------------
               Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
------------------------------------------------------------------
HINC           0.0264    0.0100   2.6477  0.0081   0.0068   0.0459
TTME_TRAIN     0.0389    0.0195   1.9916  0.0464   0.0006   0.0772
INVC_CAR      -0.0512    0.0204  -2.5103  0.0121  -0.0913  -0.0112
PSIZE_1       -0.3077    0.7317  -0.4206  0.6741  -1.7419   1.1264
PSIZE_2       -1.0800    0.6417  -1.6829  0.0924  -2.3378   0.1778
PSIZE_3       -0.7585    0.7582  -1.0004  0.3171  -2.2444   0.7275
Intercept     -1.8879    1.1138  -1.6951  0.0901  -4.0708   0.2950
=================================================================

"""

In [29]:
print("train set AUC: %.6f" % metrics.roc_auc_score(y_train, logistic.predict(X_train)))
print("test set AUC: %.6f" % metrics.roc_auc_score(y_test, logistic.predict(X_test)))

train set AUC: 0.753385
test set AUC: 0.651026


## 6. Amend model

In [30]:
X = X.drop(columns=['PSIZE_1', 'PSIZE_2', 'PSIZE_3'])

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, random_state=1234)
logistic = sm.Logit(y_train, X_train).fit()
logistic.summary2()

Optimization terminated successfully.
         Current function value: 0.521493
         Iterations 6


<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
=================================================================
Model:              Logit            Pseudo R-squared: 0.128     
Dependent Variable: y                AIC:              183.2216  
Date:               2021-07-05 15:39 BIC:              195.7174  
No. Observations:   168              Log-Likelihood:   -87.611   
Df Model:           3                LL-Null:          -100.51   
Df Residuals:       164              LLR p-value:      1.0518e-05
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     6.0000                                       
------------------------------------------------------------------
               Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
------------------------------------------------------------------
HINC           0.0266    0.0096   2.7731  0.0056   0.0078   0.0454
TTME_TRAIN     0.0335    0.0161   2.0838  0.0372   0.0020   0.0650
INVC_CAR      -0.0450    0.0168  -2.6805  0.0074  -0.0778  -0.0121
Intercept     -2.3486    0.8275  -2.8384  0.0045  -3.9704  -0.7269
=================================================================

"""

In [32]:
print("train set AUC: %.6f" % metrics.roc_auc_score(y_train, logistic.predict(X_train)))
print("test set AUC: %.6f" % metrics.roc_auc_score(y_test, logistic.predict(X_test)))

train set AUC: 0.734462
test set AUC: 0.741935
